# Spectrograms - CNN Test

In [1]:
# Import dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

import sqlalchemy
from sqlalchemy import create_engine, inspect

import math
import numpy as np
import pandas as pd
import tensorflow as tf
import keras_tuner as kt
from pprint import pprint

import os
import sys
import time
from datetime import datetime
from contextlib import redirect_stdout

%run functions.ipynb

In [2]:
# Time the run
start_time = time.time()

## Import datasets

In [3]:
# Import the data
engine = create_engine("sqlite:///voice.sqlite")

# View all of the classes
inspector = inspect(engine)
table_names = inspector.get_table_names()
table_names

['aval',
 'bval',
 'chroma1',
 'chroma10',
 'chroma11',
 'chroma12',
 'chroma2',
 'chroma3',
 'chroma4',
 'chroma5',
 'chroma6',
 'chroma7',
 'chroma8',
 'chroma9',
 'chromastd',
 'delta',
 'demographic',
 'diagnosis',
 'energy',
 'energyentropy',
 'gval',
 'habits',
 'mfcc1',
 'mfcc10',
 'mfcc11',
 'mfcc12',
 'mfcc13',
 'mfcc2',
 'mfcc3',
 'mfcc4',
 'mfcc5',
 'mfcc6',
 'mfcc7',
 'mfcc8',
 'mfcc9',
 'rval',
 'spectralcentroid',
 'spectralentropy',
 'spectralflux',
 'spectralrolloff',
 'spectralspread',
 'zcr']

In [4]:
# Initialise a dictionary to hold dataframes
dataframes = dict()

# Loop through each table
for table in table_names:
    
    # Dataframe name
    df_name = f'{table}_df'
    
    # Create dataframe
    dataframes[df_name] = pd.read_sql(
        f'SELECT * FROM {table}',
        engine
    )

## Preprocessing

### Define the target variable

In [5]:
# Isolate the diagnosis column
y = dataframes['diagnosis_df']['diagnosis'].copy()

# Encode the target variable, ignore subtype
y = y.apply(encode_binary)
y

0      0
1      0
2      1
3      1
4      1
      ..
199    0
200    1
201    1
202    0
203    0
Name: diagnosis, Length: 204, dtype: int64

### Reshape the feature variables

In [6]:
# Input shape
width_px = 225
height_px = 166
num_channels = 4 # since RGBA

# Define inputs
input_shape = (height_px, width_px, num_channels)
input_reshape = (height_px, width_px)

In [7]:
# Dataframe order
rgba_order = ['rval_df', 'gval_df', 'bval_df', 'aval_df']

# Initialise list to hold the dataframes
rgba_df_list = []

# Loop through all the dataframes
for df in rgba_order:
    
    # Define the df columns
    df_cols = dataframes[df].columns

    # Reshape to its original dimensions
    data = np.array(
        [dataframes[df][col].values.reshape(input_reshape) for col in df_cols]
    )

    # Append to the list
    rgba_df_list.append(data)

# Define the feature variables
X = np.stack(rgba_df_list, axis=-1)

# Display the first for confirmation
X[0]

array([[[ 47,  17,  99, 255],
        [ 47,  17,  99, 255],
        [ 43,  16,  93, 255],
        ...,
        [ 35,  11,  70, 255],
        [ 47,  16,  90, 255],
        [ 49,  17,  93, 255]],

       [[ 45,  17,  97, 255],
        [ 45,  17,  98, 255],
        [ 41,  16,  90, 255],
        ...,
        [ 49,  15,  93, 255],
        [ 59,  16, 108, 255],
        [ 60,  15, 111, 255]],

       [[ 46,  17,  98, 255],
        [ 46,  17,  99, 255],
        [ 43,  16,  92, 255],
        ...,
        [ 29,  13,  68, 255],
        [ 51,  16,  99, 255],
        [ 54,  16, 104, 255]],

       ...,

       [[215,  69, 107, 255],
        [215,  69, 107, 255],
        [211,  67, 109, 255],
        ...,
        [ 76,  18, 120, 255],
        [184,  56, 115, 255],
        [201,  62, 114, 255]],

       [[235,  90,  96, 255],
        [235,  90,  96, 255],
        [231,  87,  98, 255],
        ...,
        [ 91,  20, 125, 255],
        [206,  70, 105, 255],
        [225,  77, 101, 255]],

       [[250

### Split and Scale

In [8]:
# Split the preprocessed data to training and testing datasets
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y)

In [9]:
# Reshape the data
X_train_reshaped = X_train.reshape((
    X_train.shape[0], # total number of samples
    height_px * width_px * num_channels # total number flattened
))

X_test_reshaped = X_test.reshape((
    X_test.shape[0],
    height_px * width_px * num_channels
    ))

In [10]:
# Normalize training data to be between 0 and 1
X_scaler = MinMaxScaler()

# Scale the data
X_train_scaled = X_scaler.fit_transform(X_train_reshaped)
X_test_scaled = X_scaler.fit_transform(X_test_reshaped)

# Reshape the data back to the original
X_train_scaled = X_train_scaled.reshape((
    X_train_scaled.shape[0],
    height_px,
    width_px,
    num_channels
))

X_test_scaled = X_test_scaled.reshape((
    X_test_scaled.shape[0],
    height_px,
    width_px,
    num_channels
))

## Initial Test Model

__GUIDELINES__

- The number of filters for convolutional layers should:
    - Be a value to the power of 2
    - Increase for each following layer (i.e. `32`, `64`, `128`)
- The number of neurons for the fully connected layers should:
    - Be a value to the power of 2
    - Decrease for each following layer (i.e. `64`, `32`)
- Kernel size must be an odd integer

In [11]:
# Define the CNN model
cnn = Sequential()

# Add first convolutional layer
cnn.add(Conv2D(
    filters = 32,
    kernel_size = (3, 3),
    activation = 'relu',
    input_shape = (height_px, width_px, num_channels)
))

# Add first pooling layer
cnn.add(MaxPooling2D((2, 2)))

# Add second convolutional layer
cnn.add(Conv2D(
    filters = 64,
    kernel_size = (3, 3),
    activation = 'relu'
))

# Add second pooling layer
cnn.add(MaxPooling2D((2, 2)))

# Add third convolutional layer
cnn.add(Conv2D(
    filters = 128,
    kernel_size = (3, 3),
    activation = 'relu'
))

# Add third pooling layer
cnn.add(MaxPooling2D((2, 2)))

# Flatten the output before feeding into the fully connected layers
cnn.add(Flatten())

# Add dense layers for classification
cnn.add(Dense(128, activation='relu'))
cnn.add(Dense(1, activation='sigmoid'))  # Binary classification

# Display the summary
cnn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 164, 223, 32)      1184      
                                                                 
 max_pooling2d (MaxPooling2  (None, 82, 111, 32)       0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 80, 109, 64)       18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 40, 54, 64)        0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 38, 52, 128)       73856     
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 19, 26, 128)       0

In [12]:
# Compile the model
cnn.compile(
    optimizer = 'adam',
    loss = 'binary_crossentropy',
    metrics = ['accuracy']
)

In [13]:
# Train the model
cnn.fit(
    X_train_scaled,
    y_train,
    epochs = 15,
    shuffle = True, # reduce risk of overfitting
    verbose = 1
)

Epoch 1/15
5/5 [==============================] - 1s 222ms/step - loss: 1.5411 - accuracy: 0.5556
Epoch 2/15
5/5 [==============================] - 1s 222ms/step - loss: 0.6567 - accuracy: 0.7190
Epoch 3/15
5/5 [==============================] - 1s 224ms/step - loss: 0.5834 - accuracy: 0.7190
Epoch 4/15
5/5 [==============================] - 1s 241ms/step - loss: 0.6132 - accuracy: 0.7190
Epoch 5/15
5/5 [==============================] - 1s 251ms/step - loss: 0.5957 - accuracy: 0.7190
Epoch 6/15
5/5 [==============================] - 1s 228ms/step - loss: 0.5905 - accuracy: 0.7190
Epoch 7/15
5/5 [==============================] - 1s 252ms/step - loss: 0.5806 - accuracy: 0.7190
Epoch 8/15
5/5 [==============================] - 1s 226ms/step - loss: 0.5851 - accuracy: 0.7190
Epoch 9/15
5/5 [==============================] - 1s 223ms/step - loss: 0.5815 - accuracy: 0.7190
Epoch 10/15
5/5 [==============================] - 1s 224ms/step - loss: 0.5675 - accuracy: 0.7190
Epoch 11/15
5/5 [==

In [14]:
# Evaluate the model using the test data
model_loss, model_accuracy = cnn.evaluate(
    X_test_scaled,
    y_test,
    verbose = 2
)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

2/2 - 0s - loss: 0.6313 - accuracy: 0.7255 - 158ms/epoch - 79ms/step
Loss: 0.6312906742095947, Accuracy: 0.7254902124404907


## Save Results to Performance Tracker

In [15]:
# Create dictionary to save the results
results_dict = dict()

current_time = datetime.now()
results_dict['timestamp'] = current_time
results_dict['runtime'] = time.time() - start_time
results_dict['model_loss'] = model_loss
results_dict['model_accuracy'] = model_accuracy

In [16]:
# Change message
change_message = input("Changes from previous iteration: ")

# Append to results_dict
results_dict['change_message'] = change_message

Changes from previous iteration:  back to initial run to check predictions


In [17]:
# Display the dictionary
results_dict

{'timestamp': datetime.datetime(2024, 1, 6, 14, 50, 40, 488593),
 'runtime': 23.75065279006958,
 'model_loss': 0.6312906742095947,
 'model_accuracy': 0.7254902124404907,
 'change_message': 'back to initial run to check predictions'}

In [18]:
# Model summary folder path
summary_path = '../resources/tracker/cnn_summary/'
num_files = len(os.listdir(summary_path))

# Export summary to text file
with open(f'{summary_path}cnn_summary_{num_files}.txt', 'w') as f:
    with redirect_stdout(f):
        cnn.summary()
        pprint(results_dict)

In [19]:
# Convert the dictionary to a dataframe
results_df = pd.DataFrame(results_dict, index=[0])
results_df.head()

,timestamp,runtime,model_loss,model_accuracy,change_message
0,2024-01-06 14:50:40.488593,23.750653,0.631291,0.72549,back to initial run to check predictions


In [20]:
# Performance tracker
tracker_path = "../resources/tracker/cnn_performance_tracker.csv"

# Model percentage
model_pct = round(model_accuracy, 3)

# Check if the CSV exists
if os.path.exists(tracker_path):
    
    # Read the existing CSV
    tracker_df = pd.read_csv(tracker_path)
    
    # Append the new row of data
    updated_df = pd.concat([tracker_df, results_df], ignore_index=True)
    
    # Update the CSV file
    updated_df.to_csv(tracker_path, index=False)
    
    # Export model to HDF5 file
    cnn.save(f'../models/cnn/run_{len(tracker_df)}_{model_pct}.h5', save_format='h5')

else:    
    # Export to CSV
    results_df.to_csv(tracker_path, index=False)
    
    # Export model to HDF5 file
    cnn.save(f'../models/cnn/run_0_{model_pct}.h5', save_format='h5')

/Users/alyssahondrade/anaconda3/envs/dev/lib/python3.10/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


## Save Model Results to CSV

In [21]:
# Check the prediction's output probabilities
predicted_prob = cnn.predict(X_test_scaled)
clean_prob = np.round(predicted_prob, 2)

# Round to the nearest integer and flatten
clean_predicted = np.round(predicted_prob).astype(int).flatten()

# Convert to a dataframe for readability
output_prob = pd.DataFrame({
    'Actual': y_test,
    'Predicted': clean_predicted,
    'Probability': clean_prob.flatten()
})

output_prob

2/2 [==============================] - 0s 43ms/step


,Actual,Predicted,Probability
145,1,1,0.85
116,0,1,0.75
160,1,1,0.83
5,1,1,0.72
143,0,1,0.83
113,1,1,0.87
144,0,1,0.87
102,1,1,0.85
119,0,1,0.79
93,1,1,0.74
